# Validate Azure Open AI Setup

This notebook guides you through validating your Azure OpenAI environment. The sections below:

- **Install Required Packages:** Ensure all necessary Python packages are installed for Azure OpenAI and environment management.
- **Check Environment Variables:** Verify that all required Azure and OpenAI environment variables are set for secure and successful API access.
- **Utility Functions:** Provide helper functions for pretty-printing token usage and content filter results from API responses.
- **Initialize AzureOpenAI Client:** Set up the AzureOpenAI client using your environment variables for authenticated API calls.
- **Run Sample Queries:** Demonstrate how to send test prompts to your Azure OpenAI deployment and display the results, confirming your setup is working correctly.

---

## 1. Install Python Dependencies

In [3]:
# Install Required Packages
!pip install -q openai azure-identity python-dotenv --quiet

# You may need to updated your OpenAI Python library
!pip install openai --upgrade --quiet

---

## 2. Check Environment Variables


In [4]:
# Check that required environment variables are set
import os
from dotenv import load_dotenv
load_dotenv()

missing = []
required = ["AZURE_OPENAI_KEY", "AZURE_OPENAI_ENDPOINT",
            "AZURE_INFERENCE_ENDPOINT", "AZURE_AI_CONNECTION", "AZURE_OPENAI_API_VERSION",
            "GENERAL_PURPOSE", "REASONING_OLD", "REASONING_NEW",]
for var in required:
    if not os.getenv(var):
        missing.append(var)
if missing:
    raise ValueError(f"❌ Missing env vars: {missing}. Add them to .env.")

print("✅ Environment looks good: All variables are set.")

✅ Environment looks good: All variables are set.


---

## 3. Utility Function: Print Stats

In [5]:
# Pretty print some information

def print_token_and_filter_info(r):

    # Print token usage
    print(".........................")
    print("Token Costs:")
    print(f"Total Tokens: {r.usage.total_tokens}")
    print(f"Prompt Tokens: {r.usage.prompt_tokens}")
    print(f"Completion Tokens: {r.usage.completion_tokens}")
    print(f"Reasoning Tokens: {r.usage.completion_tokens_details.reasoning_tokens}")
    print(f"Output Tokens: {r.usage.total_tokens - r.usage.completion_tokens_details.reasoning_tokens}")
    print(".........................")

    # Print content filter results
    # Print content filter results
    print("Content Filter Results:")
    filter_results = getattr(r.choices[0], "content_filter_results", None)
    if filter_results is not None:
        for k, v in filter_results.items():
            print(f"{k}: {v}")
    else:
        print("No content filter results available.")
    print(".........................")

---

## 4. Utility Function" Call Chat Completion

In [6]:
from openai import AzureOpenAI
client = AzureOpenAI(
    api_key=os.environ["AZURE_OPENAI_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

def chat(model:str, prompt:str="Hello!", **kwargs):
    r = client.chat.completions.create(
        model=model,
        messages=[{"role":"user","content":prompt}],
        **kwargs
    )
    print(f"🗣️ {model} returned:")
    print(r.choices[0].message.content)

    # pretty print stats
    print_token_and_filter_info(r)
    return r


---

## 5. Test with General Purpose Model

- Observe the time taken to execute the call.
- Note the accuracy of the response
- Note the Total tokens used
- Note the Reasoning token contribution to this

In [7]:
response = chat(os.getenv("GENERAL_PURPOSE"), "How many r's in strawberry?")

🗣️ gpt-4o-mini returned:
There are three 'r's in the word "strawberry."
.........................
Token Costs:
Total Tokens: 29
Prompt Tokens: 14
Completion Tokens: 15
Reasoning Tokens: 0
Output Tokens: 29
.........................
Content Filter Results:
hate: {'filtered': False, 'severity': 'safe'}
protected_material_code: {'filtered': False, 'detected': False}
protected_material_text: {'filtered': False, 'detected': False}
self_harm: {'filtered': False, 'severity': 'safe'}
sexual: {'filtered': False, 'severity': 'safe'}
violence: {'filtered': False, 'severity': 'safe'}
.........................


---

## 6. Test With Reasoning Model (old)

- Observe the time taken to execute the call.
- Note the accuracy of the response
- Note the Total tokens used
- Note the Reasoning token contribution to this
- **How does this compare to the GPT model?**

In [8]:
response = chat(os.getenv("REASONING_OLD"), "How many r's in strawberry?")

🗣️ o1 returned:
There are three “r”s in “strawberry.”
.........................
Token Costs:
Total Tokens: 165
Prompt Tokens: 13
Completion Tokens: 152
Reasoning Tokens: 128
Output Tokens: 37
.........................
Content Filter Results:
hate: {'filtered': False, 'severity': 'safe'}
protected_material_code: {'filtered': False, 'detected': False}
protected_material_text: {'filtered': False, 'detected': False}
self_harm: {'filtered': False, 'severity': 'safe'}
sexual: {'filtered': False, 'severity': 'safe'}
violence: {'filtered': False, 'severity': 'safe'}
.........................


---

## 7. Test with Reasoning Model (new)

- Observe the time taken to execute the call.
- Note the accuracy of the response
- Note the Total tokens used
- Note the Reasoning token contribution to this
- **How does this compare to the GPT and older Reasoning models?**

In [9]:
response = chat(os.getenv("REASONING_NEW"), "How many r's in strawberry?")

🗣️ o4-mini returned:
The word “strawberry” contains three “r” letters.
.........................
Token Costs:
Total Tokens: 238
Prompt Tokens: 13
Completion Tokens: 225
Reasoning Tokens: 192
Output Tokens: 46
.........................
Content Filter Results:
hate: {'filtered': False, 'severity': 'safe'}
protected_material_code: {'filtered': False, 'detected': False}
protected_material_text: {'filtered': False, 'detected': False}
self_harm: {'filtered': False, 'severity': 'safe'}
sexual: {'filtered': False, 'severity': 'safe'}
violence: {'filtered': False, 'severity': 'safe'}
.........................


---

# Congratulations

You're all set for the labs.